In [1]:
from typing import Literal
import instructor
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Image
from pydantic import BaseModel

In [2]:
vertexai.init(location="europe-west4")

client = instructor.from_vertexai(
    client=GenerativeModel("gemini-2.0-flash"),
    mode=instructor.Mode.VERTEXAI_TOOLS,
)

In [6]:
report = Part.from_uri(
    uri="gs://genai-extraction-examples-bucket/ad_annual-report_2024_interactive.pdf",
    mime_type="application/pdf",
)

In [7]:
class AuditAnswer(BaseModel):
    question: str
    answer: str
    page: int
    citation: str

class AuditAnswers(BaseModel):
    answers: list[AuditAnswer]


client.create(
    messages=[
        (
            "Answer the following questions:"
            "- Could you specify the time frame these financial statements represent? "
            "- What accounting standards or principles were applied in the preparation of these statements? "
            "- Have there been any alterations to accounting policies or practices since prior reporting periods? "
            "- What process is used to determine and reveal major accounting estimates? "
            "- What critical assumptions and judgments were utilized in the preparation of these financial statements? "
            "- How would you describe the organization's financial standing and performance as reflected in these statements? "
            "- Do these financial results include any extraordinary or one-time items? "
            "- Are there any transactions with related parties that are documented in the financial statements? "
            "- Could you detail the principal elements of operating expenses and their classification? "
            "- Are there any asset impairments or write-downs shown in these statements? "
            "- What accounting treatment is applied to changes in fair value, particularly for financial instruments? "
            "- Do the financial statements disclose any contingent liabilities or ongoing legal issues? "
            "- Are there any post-reporting period events that might affect the financial results? "
            "- How do these statements depict the organization's liquidity and cash flow situation? "
            "- Have there been any notable shifts in working capital ratios relative to previous periods? "
            "- Could you describe the organization's debt composition and the presentation of interest expenses? "
            "- What accounting and presentation methods are used for taxes in these financial statements? "
            "- Are there any contractual commitments or obligations outlined in these statements? "
            "- Could you provide a detailed breakdown of equity, including movements in retained earnings? "
            "- If relevant, what accounting treatment is applied to foreign currency transactions and translation? "
            "- Are there any significant differences or contradictions found within these financial statements? "
            "- Could you explain the organization's approach to recognizing revenue and its implementation? "
            "- What process is used to determine and disclose provisions for doubtful accounts and bad debts? "
            "- Are there any major events or transactions that might affect the ability to compare these statements with others? "
            "- How does the organization handle risk management, and how is this communicated? "
            "- What accounting treatment is used for gains or losses from asset disposals in these statements? "
            "- Are there any non-operating revenues or expenses that influence these financial statements? "
            "- Could you describe the nature and extent of the audit or review procedures carried out on these statements? "
            "... using the attached annual statement PDF."
        ),
        report
    ],
    response_model=AuditAnswers,
)

AuditAnswers(answers=[AuditAnswer(question='Could you specify the time frame these financial statements represent?', answer='The annual report is for the year 2024.', page=1, citation='annual report 2024'), AuditAnswer(question='What accounting standards or principles were applied in the preparation of these statements?', answer='This information is not found in the document.', page=1, citation='growing together'), AuditAnswer(question='Have there been any alterations to accounting policies or practices since prior reporting periods?', answer='This information is not found in the document.', page=1, citation='growing together'), AuditAnswer(question='What process is used to determine and reveal major accounting estimates?', answer='This information is not found in the document.', page=1, citation='growing together'), AuditAnswer(question='What critical assumptions and judgments were utilized in the preparation of these financial statements?', answer='This information is not found in th